# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

# Find the ground state using direct minimisation (always using SCF is boring ...)
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.117767e+02     1.746709e+00
 * time: 0.16193199157714844
     1     1.002976e+01     9.385094e-01
 * time: 1.0209999084472656
     2    -1.150257e+01     1.110726e+00
 * time: 1.1548738479614258
     3    -3.405473e+01     8.085236e-01
 * time: 1.324098825454712
     4    -4.735618e+01     6.275138e-01
 * time: 1.4844038486480713
     5    -5.691112e+01     2.147705e-01
 * time: 1.6374528408050537
     6    -5.984240e+01     1.177397e-01
 * time: 1.7499139308929443
     7    -6.090672e+01     5.610155e-02
 * time: 1.8694698810577393
     8    -6.131512e+01     7.701744e-02
 * time: 1.9819810390472412
     9    -6.162669e+01     4.029666e-02
 * time: 2.1009960174560547
    10    -6.183832e+01     3.575361e-02
 * time: 2.214948892593384
    11    -6.198828e+01     2.145720e-02
 * time: 2.3344969749450684
    12    -6.204462e+01     1.888147e-02
 * time: 2.446462869644165
    13    -6.210862e+01     1.329417e-02
 * time: 2.566283941268

In [4]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671086
    AtomicLocal         -18.8557768
    AtomicNonlocal      14.8522700
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485401 
    Xc                  -19.3336831

    total               -62.261666456203
